## [참고]
### https://cocoa-t.tistory.com/entry/PyHessian-Loss-Landscape-%EC%8B%9C%EA%B0%81%ED%99%94-PyHessian-Neural-Networks-Through-the-Lens-of-the-Hessian

In [102]:
#!pip install pyhessian
#!pip install pytorchcv

In [103]:
%load_ext autoreload
%autoreload 2

from pyhessian import hessian
import numpy as np

from pytorchcv.model_provider import get_model as ptcv_get_model # model

import sys, os

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

import matplotlib.pyplot as plt
#%matplotlib inline 

# enable cuda devices
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from utils.parser_utils import get_args
from data import MetaLearningSystemDataLoader
from experiment_builder import ExperimentBuilder

from few_shot_learning_system import MAMLFewShotClassifier
from utils import loss_landscape_join

# 0. Dataset 지정하기

In [105]:
dataset="mini_imagenet_full_size"
# dataset="tiered_imagenet"
# dataset="CIFAR_FS"
# dataset="CUB"

title = 'miniImageNet'
# title = 'tieredImageNet'
# title = 'CIFAR-FS'
# title = 'CUB'

# 1. MAML 모델 불러오기

In [106]:
os.environ['DATASET_DIR'] = 'C:/Users/JM/PycharmProjects/MAML/datasets'
print("os.environ['DATASET_DIR'] === ", os.environ['DATASET_DIR'])

args_maml = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":dataset,
  "dataset_path":dataset,
  "reset_stored_paths":False,
  "experiment_name":"../MAML_filter128",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 100,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.0001,
  "meta_learning_rate":0.0001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":128,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,
  "samples_per_iter" : 1,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "backbone": "4-CONV",
  "arbiter": False,
  "use_bias": True,
  "smoothing": False,
  "knowledge_distillation": False,
  "momentum": "SGD"
}
)

device = torch.cuda.current_device()
args_maml.im_shape = (2, 3, args_maml.image_height, args_maml.image_width)

args_maml.use_cuda = torch.cuda.is_available()
args_maml.seed = 104
args_maml.reverse_channels=False
args_maml.labels_as_int=False
args_maml.reset_stored_filepaths=False
args_maml.num_of_gpus=1

args_maml.continue_from_epoch='latest'

os.environ['DATASET_DIR'] ===  C:/Users/JM/PycharmProjects/MAML/datasets


## 2. Arbiter 불러오기

In [107]:
os.environ['DATASET_DIR'] = 'C:/Users/JM/PycharmProjects/MAML/datasets'
print("os.environ['DATASET_DIR'] === ", os.environ['DATASET_DIR'])

args_arbiter = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":dataset,
  "dataset_path":dataset,
  "reset_stored_paths":False,
  "experiment_name":"../MAML+Arbiter_5way_5shot",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 150,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.0001,
  "meta_learning_rate":0.0001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":128,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,
  "samples_per_iter" : 1,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "backbone": "4-CONV",
  "arbiter": True,
  "use_bias": True,
  "smoothing": False,
  "knowledge_distillation": False,
  "momentum": "SGD"
}
)

device = torch.cuda.current_device()
args_arbiter.im_shape = (2, 3, args_arbiter.image_height, args_arbiter.image_width)

args_arbiter.use_cuda = torch.cuda.is_available()
args_arbiter.seed = 104
args_arbiter.reverse_channels=False
args_arbiter.labels_as_int=False
args_arbiter.reset_stored_filepaths=False
args_arbiter.num_of_gpus=1

args_arbiter.continue_from_epoch='latest'

os.environ['DATASET_DIR'] ===  C:/Users/JM/PycharmProjects/MAML/datasets


## 3. Model 구성하기

### 3.1. MAML Model 구성하기

In [108]:
# 모델을 구성한다
model_maml = MAMLFewShotClassifier(args=args_maml, device=device,
                              im_shape=(2, 3,
                                        args_maml.image_height, args_maml.image_width))

data = MetaLearningSystemDataLoader

maml_system = ExperimentBuilder(model=model_maml, data=data, args=args_maml, device=device)

Using max pooling
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
Inner Loop parameters
Outer Loop parameters
log_dir ===  C:\Users\JM\PycharmProjects\MAML\MAML_filter128
attempting to find existing checkpoint
dataset_splits ==  dict_keys(['test', 'train', 'val'])
data {'test': 12000, 'train': 38400, 'val': 9600}
train_seed 985773, val_seed: 985773, at start time
50000 50000


### 3.2.  Arbiter 구성하기

In [109]:
# 모델을 구성한다
model_arbiter = MAMLFewShotClassifier(args=args_arbiter, device=device,
                              im_shape=(2, 3,
                                        args_arbiter.image_height, args_arbiter.image_width))

data = MetaLearningSystemDataLoader

arbiter_system = ExperimentBuilder(model=model_arbiter, data=data, args=args_arbiter, device=device)

Using max pooling
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
Inner Loop parameters
Outer Loop parameters
log_dir ===  C:\Users\JM\PycharmProjects\MAML\MAML+Arbiter_5way_5shot
attempting to find existing checkpoint
dataset_splits ==  dict_keys(['test', 'train', 'val'])
data {'test': 12000, 'train': 38400, 'val': 9600}
train_seed 985773, val_seed: 985773, at start time
50000 75000


## 0. 모델 상태 확인

In [110]:
maml_system.state

{'best_val_acc': 0.6359555553396543,
 'best_val_iter': 49000,
 'current_iter': 50000,
 'best_epoch': 98,
 'train_loss_mean': 0.6403910377025605,
 'train_loss_std': 0.1255260544055785,
 'train_accuracy_mean': 0.7639866684675216,
 'train_accuracy_std': 0.06118401968549816,
 'train_loss_importance_vector_0_mean': 0.006000000052154064,
 'train_loss_importance_vector_0_std': 0.0,
 'train_loss_importance_vector_1_mean': 0.006000000052154064,
 'train_loss_importance_vector_1_std': 0.0,
 'train_loss_importance_vector_2_mean': 0.006000000052154064,
 'train_loss_importance_vector_2_std': 0.0,
 'train_loss_importance_vector_3_mean': 0.006000000052154064,
 'train_loss_importance_vector_3_std': 0.0,
 'train_loss_importance_vector_4_mean': 0.9760000109672546,
 'train_loss_importance_vector_4_std': 0.0,
 'train_learning_rate_mean': 0.00010000000000000003,
 'train_learning_rate_std': 2.710505431213761e-20,
 'val_loss_mean': 0.9505268172423045,
 'val_loss_std': 0.15325273037358733,
 'val_accuracy_mean'

In [111]:
arbiter_system.state

{'best_val_acc': 0.6818444436788559,
 'best_val_iter': 48000,
 'current_iter': 50000,
 'best_epoch': 96,
 'train_loss_mean': 0.4526471059322357,
 'train_loss_std': 0.12993412983496969,
 'train_accuracy_mean': 0.8343333342075347,
 'train_accuracy_std': 0.0548158550248203,
 'train_loss_importance_vector_0_mean': 0.006000000052154064,
 'train_loss_importance_vector_0_std': 0.0,
 'train_loss_importance_vector_1_mean': 0.006000000052154064,
 'train_loss_importance_vector_1_std': 0.0,
 'train_loss_importance_vector_2_mean': 0.006000000052154064,
 'train_loss_importance_vector_2_std': 0.0,
 'train_loss_importance_vector_3_mean': 0.006000000052154064,
 'train_loss_importance_vector_3_std': 0.0,
 'train_loss_importance_vector_4_mean': 0.9760000109672546,
 'train_loss_importance_vector_4_std': 0.0,
 'train_learning_rate_mean': 0.00010000000000000003,
 'train_learning_rate_std': 2.710505431213761e-20,
 'val_loss_mean': 0.8508474173148474,
 'val_loss_std': 0.14179470314582376,
 'val_accuracy_mean'

# 1. 학습된 모델을 불러온다

### 1.1 MAML

In [112]:
model_save_dir = maml_system.saved_models_filepath
model_name = "train_model"
model_idx = maml_system.state['best_epoch']

state = maml_system.model.load_model(model_save_dir=model_save_dir,
                                     model_name=model_name,
                                     model_idx=model_idx)

state_dict_loaded = state['network']

maml_system.model.load_state_dict(state_dict=state_dict_loaded)

# # 잘 불러왔는지 확인하는 코드
# print("state_dict_loaded == ",state_dict_loaded)
# print("="*10)
# for key, value in maml_system.model.named_parameters():
#     print(key)
#     print(value)

<All keys matched successfully>

In [113]:
# for key, value in maml_system.model.named_parameters():
#     #print(key)
#     if value.requires_grad:
#         print(key)
#         print(value.shape)

### 1.2 Arbiter

In [114]:
model_save_dir = arbiter_system.saved_models_filepath
model_name = "train_model"
model_idx = arbiter_system.state['best_epoch']

state = arbiter_system.model.load_model(model_save_dir=model_save_dir,
                                     model_name=model_name,
                                     model_idx=model_idx)

state_dict_loaded = state['network']

arbiter_system.model.load_state_dict(state_dict=state_dict_loaded)

# # 잘 불러왔는지 확인하는 코드
# print("state_dict_loaded == ",state_dict_loaded)
# print("="*10)
# for key, value in maml_system.model.named_parameters():
#     print(key)
#     print(value)

<All keys matched successfully>

# 2. Data를 불러온다

In [115]:
train_data = maml_system.data.get_train_batches(total_batches=int(600/2), augment_images=False)

In [116]:
train_sample = next(train_data)

In [117]:
x_support_set, x_target_set, y_support_set, y_target_set, seed = train_sample

x_support_set = torch.Tensor(x_support_set).float().to(device=maml_system.model.device)
x_target_set = torch.Tensor(x_target_set).float().to(device=maml_system.model.device)
y_support_set = torch.Tensor(y_support_set).long().to(device=maml_system.model.device)
y_target_set = torch.Tensor(y_target_set).long().to(device=maml_system.model.device)


x_support_set_task, y_support_set_task, x_target_set_task, y_target_set_task = next(zip(x_support_set,y_support_set,x_target_set, y_target_set))


In [118]:
y_support_set_task

tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]], device='cuda:0')

In [119]:
arbiter_x_support_set, arbiter_x_target_set, arbiter_y_support_set, arbiter_y_target_set, seed = train_sample

arbiter_x_support_set = torch.Tensor(arbiter_x_support_set).float().to(device=arbiter_system.model.device)
arbiter_x_target_set = torch.Tensor(arbiter_x_target_set).float().to(device=arbiter_system.model.device)
arbiter_y_support_set = torch.Tensor(arbiter_y_support_set).long().to(device=arbiter_system.model.device)
arbiter_y_target_set = torch.Tensor(arbiter_y_target_set).long().to(device=arbiter_system.model.device)


arbiter_x_support_set_task, arbiter_y_support_set_task, arbiter_x_target_set_task, arbiter_y_target_set_task = next(zip(arbiter_x_support_set,arbiter_y_support_set,arbiter_x_target_set, arbiter_y_target_set))


In [120]:
arbiter_y_support_set_task

tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]], device='cuda:0')

In [121]:
names_weights_copy = arbiter_system.model.get_inner_loop_parameter_dict(arbiter_system.model.classifier.named_parameters())
        
num_devices = torch.cuda.device_count() if torch.cuda.is_available() else 1

names_weights_copy = {
    name.replace('module.', ''): value.unsqueeze(0).repeat(
        [num_devices] + [1 for i in range(len(value.shape))]) for
    name, value in names_weights_copy.items()}

n, s, c, h, w = arbiter_x_target_set_task.shape

arbiter_x_support_set_task = arbiter_x_support_set_task.view(-1, c, h, w)
arbiter_y_support_set_task = arbiter_y_support_set_task.view(-1)
arbiter_x_target_set_task = arbiter_x_target_set_task.view(-1, c, h, w)
arbiter_y_target_set_task = arbiter_y_target_set_task.view(-1)

# Inner-loop (Adaptation 과정을 수행한 후, loss function을 구해야하나?)
num_steps=5
for num_step in range(num_steps):            
    support_loss, support_preds, support_loss_seperate, fetaure_map = arbiter_system.model.net_forward(
            x=arbiter_x_support_set_task,
            y=arbiter_y_support_set_task,
            weights=names_weights_copy,
            backup_running_statistics=num_step == 0,
            training=True,
            num_step=num_step,
            training_phase='test',
            epoch=0,
        )

    generated_alpha_params = {}
    
    if arbiter_system.model.args.arbiter:
        support_loss_grad = torch.autograd.grad(support_loss, names_weights_copy.values(),
                                                retain_graph=True)

        names_grads_copy = dict(zip(names_weights_copy.keys(), support_loss_grad))

        per_step_task_embedding = []

        for key, weight in names_weights_copy.items():
            weight_norm = torch.norm(weight, p=2)
            per_step_task_embedding.append(weight_norm)

        for key, grad in names_grads_copy.items():
            gradient_l2norm = torch.norm(grad, p=2)
            per_step_task_embedding.append(gradient_l2norm)

        per_step_task_embedding = torch.stack(per_step_task_embedding)

        per_step_task_embedding = (per_step_task_embedding - per_step_task_embedding.mean()) / (
                    per_step_task_embedding.std() + 1e-12)

        generated_gradient_rate = arbiter_system.model.arbiter(per_step_task_embedding)

        g = 0
        for key in names_weights_copy.keys():
            generated_alpha_params[key] = generated_gradient_rate[g]
            g += 1

    names_weights_copy,names_grads_copy = arbiter_system.model.apply_inner_loop_update(loss=support_loss,
                                                                      support_loss_seperate=support_loss_seperate,
                                                      names_weights_copy=names_weights_copy,
                                                      alpha=generated_alpha_params,
                                                      use_second_order=args_arbiter.second_order,
                                                      current_step_idx=num_step,
                                                      current_iter=arbiter_system.state['current_iter'],
                                                      training_phase='test')


for name, param in arbiter_system.model.classifier.named_parameters():
    if param.requires_grad:
        if "norm_layer" not in name:
            param.data = names_weights_copy[name].squeeze().to(device=device)

In [122]:
names_weights_copy = maml_system.model.get_inner_loop_parameter_dict(maml_system.model.classifier.named_parameters())
        
        
num_devices = torch.cuda.device_count() if torch.cuda.is_available() else 1

names_weights_copy = {
    name.replace('module.', ''): value.unsqueeze(0).repeat(
        [num_devices] + [1 for i in range(len(value.shape))]) for
    name, value in names_weights_copy.items()}

n, s, c, h, w = x_target_set_task.shape

x_support_set_task = x_support_set_task.view(-1, c, h, w)
y_support_set_task = y_support_set_task.view(-1)
x_target_set_task = x_target_set_task.view(-1, c, h, w)
y_target_set_task = y_target_set_task.view(-1)

# Inner-loop (Adaptation 과정을 수행한 후, loss function을 구해야하나?)
num_steps=5
for num_step in range(num_steps):            
    support_loss, support_preds, support_loss_seperate, fetaure_map = maml_system.model.net_forward(
            x=x_support_set_task,
            y=y_support_set_task,
            weights=names_weights_copy,
            backup_running_statistics=num_step == 0,
            training=True,
            num_step=num_step,
            training_phase='test',
            epoch=0,
        )

    generated_alpha_params = {}


    names_weights_copy,names_grads_copy = maml_system.model.apply_inner_loop_update(loss=support_loss,
                                                                   support_loss_seperate=support_loss_seperate,
                                                      names_weights_copy=names_weights_copy,
                                                      alpha=generated_alpha_params,
                                                      use_second_order=args_maml.second_order,
                                                      current_step_idx=num_step,
                                                      current_iter=maml_system.state['current_iter'],
                                                      training_phase='test')


for name, param in maml_system.model.classifier.named_parameters():
    if param.requires_grad:
        if "norm_layer" not in name:
            param.data = names_weights_copy[name].squeeze().to(device=device) 

## landscape 함수 그리기

inputs shape==  torch.Size([25, 3, 84, 84])
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
tensor([[[[-1.1935e-04,  2.6568e-03,  6.7283e-03],
          [-7.2114e-05,  1.0739e-03,  4.9787e-03],
          [-8.9697e-04,  1.1596e-03,  3.2536e-03]],

         [[-1.4466e-03,  7.7378e-04,  3.5977e-03],
          [-1.7130e-03, -1.1535e-03,  1.4853e-03],
          [-3.0795e-03, -1.7881e-03, -6.2774e-04]],

         [[ 2.4235e-03,  4.0640e-03,  6.5466e-03],
          [ 1.4213e-03,  1.7149e-03,  4.3617e-03],
          [-7.5947e-04,  2.3111e-04,  1.2487e-03]]],


        [[[-7.8490e-03, -7.4451e-03, -5.3934e-03],
          [-6.7272e-03, -7.8326e-03, -5.3003e-03],
          [-5.4965e-03, -6.9102e-03, -4.7017e-03]],

         [[-5.1551e-03, -4.8080e-03, -2.4748e-03],
          [-3.1338e-03, -4.1795e-03, -1.4726e-03],
          [-1.4338e-03, -2.7918e-03, -6.6917e-04]],

         [[

tensor([ 5.5818e-06,  6.9003e-07,  1.4954e-05, -2.5770e-06,  2.2499e-07,
        -5.2190e-06,  1.4669e-05,  3.5707e-06,  5.9871e-06,  7.8576e-06,
         2.2586e-06,  8.5187e-06,  4.3407e-06, -9.2927e-06, -2.5173e-06,
         2.6697e-06,  2.8163e-06,  1.0631e-05, -6.0742e-07, -8.6829e-06,
         2.3265e-06, -4.8669e-06, -2.1020e-06, -9.1709e-06, -9.1863e-06,
        -1.4261e-05, -5.0590e-06, -2.9776e-06, -4.1102e-06, -7.3416e-06,
        -5.8609e-06,  5.1562e-06, -1.1074e-05,  8.5053e-06, -1.0351e-05,
         1.2303e-05, -6.4173e-06,  1.6817e-05,  1.1594e-05, -9.8496e-06,
        -7.2978e-06,  2.5708e-06, -6.5265e-06, -1.1957e-06,  8.7703e-06,
        -8.6540e-06, -3.0323e-06,  7.0137e-06,  1.7872e-05, -1.1028e-06,
         8.2166e-06, -6.6537e-06,  6.5627e-06,  3.2491e-06,  1.0241e-06,
        -6.8898e-06, -1.6842e-05,  3.6464e-06,  9.5761e-06,  6.9629e-06,
         1.0244e-05,  6.7453e-06,  6.1995e-06,  1.0857e-05, -8.4974e-07,
        -2.9949e-06, -7.0920e-06,  1.1113e-05, -1.5

No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop params
No inner loop

In [123]:
ls = loss_landscape_join.landscape(maml_system.model.classifier, arbiter_system.model.classifier, args_arbiter)
ls.show_2djoin(x_support_set_task, y_support_set_task, title=title)